In [ ]:
import random
import json
import queue

class SimulationEngine:
    def __init__(self, lots, machines, recipe_queues, max_simulation_time):
        self.lots = lots  # Dict {lot_id: Lot instance}
        self.machines = machines  # Dict {machine_id: Machine instance}
        self.recipe_queues = recipe_queues # Dict {recp_id: recp PQ}
        self.general_fel = queue.PriorityQueue()  # Global system event queue
        self.currsimtime = 0  # Current simulation time
        self.max_simulation_time = max_simulation_time  # Stop condition

    def assign_lots_to_machines(self, machine_id):
        """main strategy: Assign lots to machines """
        for machine in self.machines.values():
            if not machine.canServe():
                continue

            available_recipes = [r for r in self.recipe_queues if not self.recipe_queues[r].empty()]
            if not available_recipes:
                continue

            best_recipe = min(available_recipes, key=lambda r: self.recipe_queues[r].queue[0][0])  # Lowest ACR
            _, lot_id = self.recipe_queues[best_recipe].get()
            lot = self.lots[lot_id]

            # Generate a random recipe processing time
            processing_time = random.randint(5, 15)
            finish_time = self.currsimtime + processing_time

            machine.assign_lot(lot, finish_time)
            lot.FEL.put((finish_time, machine.machine_id, best_recipe, lot.ACR))
            
            self.general_fel.put((finish_time, f"Assigned {lot_id} to {machine.machine_id} for {best_recipe} at {self.currsimtime}"))

    def process_events(self):
        """Main event loop"""
        while self.currsimtime < self.max_simulation_time:
            if self.general_fel.empty():
                break

            next_event_time, event_msg = self.general_fel.get()
            if next_event_time >= self.max_simulation_time:
                break

            self.currsimtime = next_event_time
            print(event_msg)  # Log events in real-time

            for machine in self.machines.values():
                if not machine.FEL.empty() and machine.FEL.queue[0][0] == self.currsimtime:
                    finish_time, lot_id, recipe = machine.complete_recipe()
                    self.general_fel.put((self.currsimtime, f"Completed {recipe} for {lot_id} on {machine.machine_id}"))

                    lot = self.lots[lot_id]
                    lot.update_acr()  # Update urgency
                    lot.next_recipe()

                    if lot.curr_recipe:
                        self.recipe_queues[lot.curr_recipe].put((lot.ACR, lot.lot_id))

                    machine.state = "ready"
                    self.assign_lots_to_machines()  # Reassign new lots

            self.recalculate_acr()  # Periodic ACR recalculation

    def recalculate_acr(self):
        """Recalculate ACR for lots still in queues periodically"""
        if self.currsimtime % 10 == 0:  # Example: Recalculate every 10 time units
            for recipe, queue_pq in self.recipe_queues.items():
                temp_list = []
                while not queue_pq.empty():
                    acr, lot_id = queue_pq.get()
                    lot = self.lots[lot_id]
                    lot.update_acr()
                    temp_list.append((lot.ACR, lot_id))  # Update ACR in queue

                for item in temp_list:
                    queue_pq.put(item)  # Reinsert with updated values

    def export_results(self):
        """Export FELs to JSON"""
        with open("machine_fel.json", "w") as f:
            json.dump({m_id: list(m.FEL.queue) for m_id, m in self.machines.items()}, f, indent=4)

        with open("lot_fel.json", "w") as f:
            json.dump({l_id: list(l.FEL.queue) for l_id, l in self.lots.items()}, f, indent=4)

        with open("general_fel.json", "w") as f:
            json.dump(list(self.general_fel.queue), f, indent=4)


# Initialize and Run Simulation
lots = {
    f"Lot-{i+1}": Lot(f"Lot-{i+1}", f"Part-{i%5}", ["Recipe-1", "Recipe-2", "Recipe-3"]) for i in range(10)
}
machines = {
    f"Machine-{i+1}": Machine(f"Machine-{i+1}", ["Recipe-1", "Recipe-2", "Recipe-3"]) for i in range(5)
}

engine = SimulationEngine(lots, machines, max_simulation_time=100)
engine.initialize_queues()
engine.assign_lots_to_machines()
engine.process_events()
engine.export_results()
